### Imports & Environment Variables 

In [ ]:
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core import Settings

from llama_index.readers.file import PagedCSVReader
from llama_index.core.ingestion import IngestionPipeline

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")

### CSV File Structure and Use Case
The CSV file contains dummy customer data, comprising various attributes like first name, last name, company, etc. This dataset will be utilized for a RAG use case, facilitating the creation of a customer information Q&A system.

In [ ]:
import pandas as pd

file_path = ('./data/unspsc_data.csv')
data = pd.read_csv(file_path)
# data['Segment'] = data['Segment'].astype("Int64")
# data['Family'] = data['Family'].astype("Int64")
# data['Class'] = data['Class'].astype("Int64")
# data['Commodity'] = data['Commodity'].astype("Int64")
# data.head()

In [ ]:
def show_hierarchy(commodity_code: str):
    """
    show the hierarchy in pretty format
    input: 
        43000000,Information Technology Broadcasting and Telecommunications," communications and computer equipment as well as peripheral input, output and storage devices.  networking equipment and general computer components and supplies such as printers, electronic storage media and connection cables.",43210000.0,Computer Equipment and Accessories,"The computer and it's accessories together provide a platform to enable user computing. A computer is a device that accepts information (in the form of digital data) and manipulates it for some result based on a program or sequence of instructions on how data is to be processed. Complex computers also include the means for storing data (including the program, which is also a form of data) for some necessary duration. A program may be invariable and built into the computer (and called logic circuitry as it is on microprocessors) or different programs may be provided to the computer (loaded into its storage and then started by an administrator or user). Today's computers have both kinds of programming.",43211900.0,Computer displays,"A display is a computer output surface or a screen and a projecting mechanism that shows text and graphic images. In some computers, the display is packaged in a separate unit called a monitor. In notebook computers, the display is integrated into a unit. Also called video display terminals VDTs, a display uses a cathode ray tube (CRT), liquid crystal display (LCD), light-emitting diode, gas plasma, or other image projection technology.",43211902.0,Liquid crystal display LCD panels or monitors,Flat panel diaplays are Liquid Crystal Display monitors.They use liquid displays instead of Cathod Ray guns.
    out:
        43000000 > 43210000 > 43211900 > 43211902
        Information Technology Broadcasting and Telecommunications > Computer Equipment and Accessories > Computer displays > Liquid crystal display LCD panels or monitors
    """
    d = data[data['Commodity'] == commodity_code]
    cols_to_first_line = ['Segment','Family','Class','Commodity']
    cols_to_second_line = ['Segment Title','Family Title','Class Title','Commodity Title']
    cols_to_third_line = ['Segment Definition','Family Definition','Class Definition','Commodity Definition']
    for i in range(len(d)):
        print(f"{d.iloc[i][cols_to_first_line[0]]} > {d.iloc[i][cols_to_first_line[1]]} > {d.iloc[i][cols_to_first_line[2]]} > {d.iloc[i][cols_to_first_line[3]]}")
        print(f"{d.iloc[i][cols_to_second_line[0]]} > {d.iloc[i][cols_to_second_line[1]]} > {d.iloc[i][cols_to_second_line[2]]} > {d.iloc[i][cols_to_second_line[3]]}")
        print(f"{d.iloc[i][cols_to_third_line[0]]} > {d.iloc[i][cols_to_third_line[1]]} > {d.iloc[i][cols_to_third_line[2]]} > {d.iloc[i][cols_to_third_line[3]]}")



### Vector Store

In [ ]:
# Create Pinecone Vector Store to store embeddings
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_4GbJvr_HYNMj3xJ7xFJNWguGB1VifdeD7P3mX25YjsWuzkTxwTawFyxq5PrqjAnjQiHBQW")

In [ ]:
# Create index if not exists
# from pinecone import ServerlessSpec
# pc.create_index(
#     name="unspsc-data",
#     dimension=1024,
#     metric="dotproduct",
#     spec=ServerlessSpec(cloud="aws", region="us-east-1"),
# )

In [23]:
from llama_index.vector_stores.pinecone import PineconeVectorStore

pinecone_index = pc.Index("unspsc-data")
vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index,
    add_sparse_vector=True,
)

### Load and Process CSV Data as Document

In [ ]:
csv_reader = PagedCSVReader()

reader = SimpleDirectoryReader( 
    input_files=[file_path],
    file_extractor= {".csv": csv_reader}
)

docs = reader.load_data()

#### Ingestion Pipeline

In [ ]:
pipeline = IngestionPipeline(
    # vector_store=vector_store,
    documents=docs
)

In [ ]:
nodes = pipeline.run(show_progress=True)

In [ ]:
# By inserting nodes to vector store with manual push, we can access the nodes with embeddings, even if we get an error when pushing to vector store
nodes_with_embeddings = [n for n in nodes if n.embedding is not None]
if nodes_with_embeddings:
    vector_store.add(nodes_with_embeddings)
else:
    print("No nodes with embeddings found")

### Vector Store Index

In [24]:
from llama_index.core import VectorStoreIndex

vector_store_index = VectorStoreIndex.from_vector_store(vector_store)

In [25]:
from llama_index.core import PromptTemplate


SYSTEM_PROMPT = """You are a UNSPSC classification expert that strictly uses provided context. Follow these steps:

1. CONTEXT ANALYSIS:
- Thoroughly analyze provided UNSPSC code context
- Identify hierarchy levels: Segment > Family > Class > Commodity

2. Output Format should be in JSON format:
Example of Success Response:
{
    "success": true,
    "unspsc_code": {code},
    "hierarchy": [
        {
            "segment_code": {segment_code},
            "segment_name": {segment_name}
        },
        {
            "family_code": {family_code},
            "family_name": {family_name}
        },
        {
            "class_code": {class_code},
            "class_name": {class_name}
        },
        {
            "commodity_code": {commodity_code},
            "commodity_name": {commodity_name}
        }
    ],
    "description": {description},
    "enriched_description": {enriched_description}
}
Or Failure Response:
{
    "success": false,
    "error": "I couldn't find a matching UNSPSC code.",
    "error_reason": "The product description does not match any of the UNSPSC codes in the context.",
}
"""
# 2. DESCRIPTION ENRICHMENT:
# - Expand product description using context terms
# - Maintain original meaning
# - Add technical/synonym terms from context

# 3. CLASSIFICATION:
# - Search for EXACT match in this order: Commodity > Class > Family > Segment
# - NEVER invent codes not in context
# - If no match, return enriched description

# 4. OUTPUT FORMAT:
# [SUCCESS RESPONSE]
# UNSPSC Code: {code}
# Hierarchy:
# - Segment {segment_code}: {segment_name}
# - Family {family_code}: {family_name}
# - Class {class_code}: {class_name}
# - Commodity {commodity_code}: {commodity_name}

# [FAILURE RESPONSE]
# I couldn't find a matching UNSPSC code.
# Enriched Description: {enriched_description}
# Please try with this enriched version.

# 5. SAFETY:
# - If uncertain, default to failure response
# - Never guess codes
# - Flag ambiguous cases
# """

query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST] "
)


In [26]:
# Ollama LLM
from llama_index.llms.ollama import Ollama

llm = Ollama(
    model="llama3.1:8b",
    temperature=0.7,
    max_tokens=512,
    context_window=4096,
    request_timeout=300.0,
    system_prompt=SYSTEM_PROMPT,
)


In [ ]:
# from llama_index.llms.openrouter import OpenRouter

# # OpenRouter LLM
# llm = OpenRouter(
#     api_key="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",
#     max_tokens=512,
#     context_window=4096,
#     temperature=0.7,
#     # model="cognitivecomputations/dolphin3.0-r1-mistral-24b:free",
#     model="deepseek/deepseek-r1:free",
#     query_wrapper_prompt=query_wrapper_prompt,
#     # system_prompt=SYSTEM_PROMPT # ? do we need this?
# )

In [ ]:
from llama_index.llms.groq import Groq

# OpenRouter LLM
llm = Groq(
    api_key="API_KEY",
    max_tokens=512,
    context_window=4096,
    temperature=0,
    # model="cognitivecomputations/dolphin3.0-r1-mistral-24b:free",
    model="llama-3.3-70b-versatile",
    query_wrapper_prompt=query_wrapper_prompt,
    system_prompt=SYSTEM_PROMPT # ? do we need this?
)

In [ ]:
from pydantic import BaseModel


class UNSPSCResponse(BaseModel):
    """Data model for a UNSPSC response."""

    unspsc_code: str
    hierarchy: list[dict[str, str]]
    description: str
    enriched_description: str
    error: str
    error_reason: str

In [27]:
query="2-Inch Galvanized Steel Pipe Test Plug with Flange and Wing Nut Closure, Featuring Molded Rubber Gasket"

In [40]:
query_engine = vector_store_index.as_query_engine(
    similarity_top_k=3,
    llm=llm,
    verbose=False,
    # response_model=UNSPSCResponse,
    # response_model_kwargs={"unspsc_code": "UNSPSC Code", "hierarchy": "Hierarchy", "description": "Description"},
    # response_mode=""
)

### Query the rag bot with a question based on the CSV data

In [45]:
# response = query_engine.query("What is the commodity code for 'Medical unit'")
# response = query_engine.query("inner diameter water separator cartridge for fuel applications, 98.7% efficient at up to 24 GPM flow rate, glass/paper element.")
response = query_engine.query("A cartridge filter-water separator for fuel")


In [42]:
response = query_engine.query(query)

In [46]:
print(response.response)

{
    "success": true,
    "unspsc_code": "25202600",
    "hierarchy": [
        {
            "segment_code": "25000000",
            "segment_name": "Vehicles and their Accessories and Components"
        },
        {
            "family_code": "25200000",
            "family_name": "Aerospace systems and components and equipment"
        },
        {
            "class_code": "25202600",
            "class_name": "Aircraft environmental control systems and components"
        },
        {
            "commodity_code": "25202610",
            "commodity_name": "Microfilters"
        }
    ],
    "description": "Aircraft environmental control systems and components",
    "enriched_description": "Microfilters are used as pre-filters for the efficient and continuous removal of solids such as rust, sand and other particulates from aviation fuels."
}


In [44]:
print(response.response)

{
    "success": true,
    "unspsc_code": "31402001",
    "hierarchy": [
        {
            "segment_code": "31000000",
            "segment_name": "Manufacturing Components and Supplies"
        },
        {
            "family_code": "31400000",
            "family_name": "Gaskets"
        },
        {
            "class_code": "31402000",
            "class_name": "Miscellaneous gaskets"
        },
        {
            "commodity_code": "31402001",
            "commodity_name": "General jointing gasket"
        }
    ],
    "description": "A jointing gasket is a type of gasket that is used to provide a tight seal between two mating surfaces that have irregular or non-parallel surfaces.",
    "enriched_description": "The given product, 2-Inch Galvanized Steel Pipe Test Plug with Flange and Wing Nut Closure, Featuring Molded Rubber Gasket, utilizes a molded rubber gasket to provide a tight seal, which aligns with the characteristics of a general jointing gasket."
}


In [32]:
print(response.response)

{
    "success": true,
    "unspsc_code": {commodity_code},
    "hierarchy": [
        {
            "segment_code": {segment_code},
            "segment_name": {segment_name}
        },
        {
            "family_code": {family_code},
            "family_name": {family_name}
        },
        {
            "class_code": {class_code},
            "class_name": {class_name}
        },
        {
            "commodity_code": {commodity_code},
            "commodity_name": {commodity_name}
        }
    ],
    "description": "A filtering device used in fuel systems.",
    "enriched_description": "The filtered water is removed from the fuel, preventing contamination."
}
{
    "hierarchy": [
        {
            "segment_code": "25000000",
            "segment_name": "Vehicles and their Accessories and Components"
        },
        {
            "family_code": "25200000",
            "family_name": "Aerospace systems and components and equipment"
        },
        {
            "clas

'{\n    "success": true,\n    "unspsc_code": {\n        "segment_code": "43000000",\n        "family_code": "43210000",\n        "class_code": "43211900",\n        "commodity_code": "43211905"\n    },\n    "hierarchy": [\n        {\n            "segment_code": "43000000",\n            "segment_name": "Information Technology Broadcasting and Telecommunications"\n        },\n        {\n            "family_code": "43210000",\n            "family_name": "Computer Equipment and Accessories"\n        },\n        {\n            "class_code": "43211900",\n            "class_name": "Computer displays"\n        },\n        {\n            "commodity_code": "43211905",\n            "commodity_name": "Organic light emitting displays"\n        }\n    ],\n    "description": "A display is a computer output surface or a screen and a projecting mechanism that shows text and graphic images.",\n    "enriched_description": "An organic light emitting displau (OLED) is an electronic device made by placing a series of organic thin films between two conductors. When electrical current is applied, a bright light is emitted."\n}'

In [ ]:
show_hierarchy(40141771)
show_hierarchy(40161513)

# FILTER,CARTRIDGE: WATER SEPARATOR,FUEL,3.866IN ID,4.575IN OD,12.323IN LG,10U,22 TO 24 GPM,98.7%,GLASS/PAPER ELEMENT,1-1/4IN-12 UNS-2B SPIN-ON,SAE J1985,SIZE: 4.736IN OD SEAM,W/ GASKET
